In [1]:
pip install opencv-python opencv-contrib-python scikit-image mahotas


  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached mahotas-1.4.18-cp311-cp311-win_amd64.whl.metadata (14 kB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Using cached mahotas-1.4.18-cp311-cp311-win_amd64.whl (1.7 MB)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\siddh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

# Load the CSV file containing labels
df = pd.read_csv('label.csv')
allClasses=df['label'].unique()
imagesPerClass = df['label'].value_counts()

In [32]:
df

,filename,label,hog_features
0,Image_1.jpg,sitting,"[0.2602150976933022, 0.155749659652248, 0.0978..."
1,Image_2.jpg,using_laptop,"[0.15454383002193137, 0.0719306605013999, 0.07..."
2,Image_3.jpg,hugging,"[0.005943894256425121, 0.0, 0.0016811870981394..."
3,Image_4.jpg,sleeping,"[0.08655324940988982, 0.13651734151945993, 0.0..."
4,Image_5.jpg,using_laptop,"[0.12369862424069092, 0.07044763837149698, 0.0..."
...,...,...,...
12595,Image_12596.jpg,sitting,"[0.15301149047520893, 0.05533839192311787, 0.1..."
12596,Image_12597.jpg,clapping,"[0.24126482758151405, 0.043683636772690375, 0...."
12597,Image_12598.jpg,sitting,"[0.02833319162984024, 0.026214920858776514, 0...."
12598,Image_12599.jpg,dancing,"[0.10675965264418201, 0.00876285842280374, 0.2..."


In [37]:
import pandas as pd

# Step 2: Check for missing values in the dataset
missing_values = df.isnull().sum()


Missing values in each column:
filename    0
label       0
dtype: int64
No missing values in the dataset.


In [40]:
pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\siddh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog, local_binary_pattern
from skimage.color import rgb2gray
from skimage.filters import gabor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report
import cv2
from skimage import filters
# https://github.com/henrhoi/image-classification/blob/master/feature_extraction_and_exploratory_data_analysis.ipynb
# https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/
# https://medium.com/@girishajmera/hog-histogram-of-oriented-gradients-an-amazing-feature-extraction-engine-for-medical-images-5a2203b47ccd
# https://www.geeksforgeeks.org/hog-feature-visualization-in-python-using-skimage/
# Load Dataset

def preprocess_image(image, target_size=(170, 170)):
    image_resized = np.array(Image.fromarray(image).resize(target_size))
    return image_resized

def extract_hog_features(image):
    image_gray = rgb2gray(image)
    features, _ = hog(image_gray, block_norm='L2-Hys', pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True)
    return features
# https://www.geeksforgeeks.org/visualizing-colors-in-images-using-histogram-in-python/
# https://stackoverflow.com/questions/57385640/how-to-extract-color-features-via-histogram-from-a-masked-image
def extract_color_histogram(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hist = cv2.calcHist([image_rgb], [0, 1, 2], None, [16, 16, 16], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist
# https://thepythoncode.com/article/sift-feature-extraction-using-opencv-in-python
# https://www.geeksforgeeks.org/sift-interest-point-detector-using-python-opencv/
def extract_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is not None:
        return descriptors.flatten()[:128]  
    return np.zeros(128)  
# https://www.geeksforgeeks.org/create-local-binary-pattern-of-an-image-using-opencv-python/

def extract_lbp_features(image):
    image_gray = rgb2gray(image)
    lbp = local_binary_pattern(image_gray, P=8, R=1, method='uniform')  
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp, bins=n_bins, range=(0, n_bins), density=True)
    return hist
# https://pyimagesearch.com/2015/12/07/local-binary-patterns-with-python-opencv/
def extract_canny_edges(image):
    edges = cv2.Canny(image, threshold1=100, threshold2=200)
    return edges.flatten()

hogFeatures=[]
histoFeatures=[]
lbpFeatures=[]
siftFeatures=[]
image_labels = []
cany=[]
def extract_combined_features(image):
    processed_image = preprocess_image(image)
    chog_features = extract_hog_features(processed_image)
    ccolor_histogram = extract_color_histogram(processed_image)
    clbp_features = extract_lbp_features(processed_image)
    hogFeatures.append(chog_features)
    histoFeatures.append(ccolor_histogram)
    lbpFeatures.append(clbp_features)
    cany.append(extract_canny_edges(processed_image))
    siftFeatures.append(extract_sift_features(processed_image))
for idx, row in df.iterrows():
    image_name = row['filename']
    label = row['label']
    image_path = os.path.join('data', image_name)
    image = np.array(Image.open(image_path))
    extract_combined_features(image)
    image_labels.append(label)



C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [5]:
# import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog, local_binary_pattern
from skimage.color import rgb2gray
from skimage.filters import gabor
from skimage import filters
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report
import cv2
def extract_sobel_edges(image):
    image_gray = rgb2gray(image)
    edge_sobel = filters.sobel(image_gray)
    return edge_sobel.flatten()
sobelEdgeFeatures = []
for idx, row in df.iterrows():
    image_name = row['filename']
    label = row['label']
    image_path = os.path.join('data', image_name)
    image = np.array(Image.open(image_path))
    processed_image = preprocess_image(image)
    csobel_edge = extract_sobel_edges(processed_image)
    sobelEdgeFeatures.append(csobel_edge)


In [15]:
hogFeatures_np = np.array(hogFeatures)
histoFeatures_np = np.array(histoFeatures)
lbpFeatures_np = np.array(lbpFeatures)
cannyFeatures_np=np.array(cany)
siftFeatures_np=np.array(siftFeatures)
df['hog_features'] = list(hogFeatures_np)
df['color_histogram'] = list(histoFeatures_np)
df['lbp_features'] = list(lbpFeatures_np)
df['cany_features'] = list(cannyFeatures_np)
df['sift_features'] = list(siftFeatures_np)
df['label']=list(image_labels)

In [7]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
import cv2
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
X_train, X_test, y_train, y_test = train_test_split(np.concatenate([hogFeatures_np, histoFeatures_np], axis=1), image_labels, test_size=0.2, random_state=42)
for n_components in range(50, 301, 50):
    print(f"Applying PCA with {n_components} components")
    pca = PCA(n_components=n_components, random_state=42)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    perceptron_model = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
    perceptron_model.fit(X_train_pca, y_train)
    y_pred = perceptron_model.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Perceptron Model Accuracy with {n_components} PCA components: {accuracy}")


Applying PCA with 50 components
Perceptron Model Accuracy with 50 PCA components: 0.1357142857142857

------------------------------------------------------------

Applying PCA with 100 components
Perceptron Model Accuracy with 100 PCA components: 0.15674603174603174

------------------------------------------------------------

Applying PCA with 150 components
Perceptron Model Accuracy with 150 PCA components: 0.18968253968253967

------------------------------------------------------------

Applying PCA with 200 components
Perceptron Model Accuracy with 200 PCA components: 0.19563492063492063

------------------------------------------------------------

Applying PCA with 250 components
Perceptron Model Accuracy with 250 PCA components: 0.19642857142857142

------------------------------------------------------------

Applying PCA with 300 components
Perceptron Model Accuracy with 300 PCA components: 0.2123015873015873

------------------------------------------------------------



In [11]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(np.concatenate([hogFeatures_np, histoFeatures_np], axis=1), image_labels, test_size=0.2, random_state=42)
perceptron_model = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
perceptron_model.fit(X_train, y_train)
y_pred_perceptron = perceptron_model.predict(X_test)
accuracy_perceptron = accuracy_score(y_test, y_pred_perceptron)
print(f"Perceptron Model Accuracy: {accuracy_perceptron}")
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Naive Bayes Model Accuracy: {accuracy_nb}")
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree Model Accuracy: {accuracy_dt}")


Perceptron Model Accuracy: 0.23095238095238096
Classification Report for Perceptron:
                    precision    recall  f1-score   support

           calling       0.22      0.05      0.08       173
          clapping       0.24      0.34      0.28       160
           cycling       0.29      0.44      0.35       186
           dancing       0.38      0.19      0.26       180
          drinking       0.10      0.14      0.12       152
            eating       0.48      0.17      0.25       151
          fighting       0.47      0.23      0.31       186
           hugging       0.15      0.15      0.15       151
          laughing       0.28      0.28      0.28       179
listening_to_music       0.18      0.07      0.10       176
           running       0.33      0.35      0.34       155
           sitting       0.22      0.26      0.24       163
          sleeping       0.39      0.17      0.24       162
           texting       0.12      0.36      0.18       183
      using_la

In [34]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import cv2
import xgboost as xgb
image_labels=np.array(image_labels)
X_train, X_test, y_train, y_test = train_test_split(np.concatenate([hogFeatures_np, histoFeatures_np,lbpFeatures_np], axis=1), image_labels, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
from sklearn.ensemble import AdaBoostClassifier
ada_model = AdaBoostClassifier(n_estimators=300, random_state=42)
ada_model.fit(X_train, y_train)
y_pred_ada = ada_model.predict(X_test)
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print(f"AdaBoost Model Accuracy: {accuracy_ada}")
print("AdaBoost Classification Report:")
print(classification_report(y_test, y_pred_ada))




C:\Users\siddh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Model Accuracy: 0.25277777777777777
AdaBoost Classification Report:
                    precision    recall  f1-score   support

           calling       0.19      0.20      0.19       173
          clapping       0.16      0.12      0.14       160
           cycling       0.47      0.49      0.48       186
           dancing       0.35      0.39      0.37       180
          drinking       0.08      0.07      0.07       152
            eating       0.43      0.55      0.48       151
          fighting       0.35      0.39      0.37       186
           hugging       0.19      0.13      0.16       151
          laughing       0.25      0.42      0.31       179
listening_to_music       0.13      0.09      0.11       176
           running       0.29      0.29      0.29       155
           sitting       0.15      0.13      0.14       163
          sleeping       0.18      0.18      0.18       162
           texting       0.13      0.09      0.11       183
      using_laptop    

In [29]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import cv2

image_labels=np.array(image_labels)
combined_features = np.concatenate([hogFeatures_np, histoFeatures_np,lbpFeatures_np], axis=1)
X_train, X_test, y_train, y_test = train_test_split(combined_features, image_labels, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
rf_model = RandomForestClassifier(random_state=42,n_estimators=300,min_samples_leaf=2,min_samples_split=10)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))



Model Accuracy: 0.33690476190476193
Classification Report:
                    precision    recall  f1-score   support

           calling       0.25      0.19      0.22       173
          clapping       0.32      0.22      0.26       160
           cycling       0.39      0.60      0.47       186
           dancing       0.48      0.51      0.49       180
          drinking       0.20      0.12      0.15       152
            eating       0.32      0.76      0.45       151
          fighting       0.36      0.49      0.41       186
           hugging       0.28      0.12      0.17       151
          laughing       0.35      0.39      0.37       179
listening_to_music       0.26      0.13      0.17       176
           running       0.40      0.34      0.37       155
           sitting       0.34      0.11      0.17       163
          sleeping       0.37      0.41      0.39       162
           texting       0.28      0.22      0.25       183
      using_laptop       0.29      0.40 

In [16]:
import pickle
import xgboost as xgb

with open('RandomForest_36_hog_histo_lbp.pkl', 'wb') as file:
    pickle.dump(rf_model, file)

NameError: name 'rf_model' is not defined

In [17]:
with open('RandomForest_36_hog_histo_lbp.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
    y_pred = loaded_model.predict(X_test)

# Accuracy and Classification Report
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy}")

EOFError: Ran out of input

In [72]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import cv2
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(combined_features, image_labels)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300} :30
# {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300} : 30 smote pca
# {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300} : 0.34563492063492063 smote 
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_rf_model = grid_search.best_estimator_
y_pred = best_rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(grid_search.best_params_)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Model Accuracy: 0.34563492063492063
Classification Report:
                    precision    recall  f1-score   support

           calling       0.23      0.21      0.22       173
          clapping       0.38      0.29      0.33       160
           cycling       0.43      0.61      0.50       186
           dancing       0.48      0.42      0.45       180
          drinking       0.26      0.11      0.16       152
            eating       0.34      0.81      0.47       151
          fighting       0.39      0.52      0.44       186
           hugging       0.22      0.13      0.16       151
          laughing       0.33      0.34      0.34       179
listening_to_music       0.34      0.18      0.24       176
           running       0.40      0.33      0.36       155
           sitting       0.29      0.17      0.21       163
          sleeping       0.37      0.44      0.40       162
           texting       0.30      0.2

In [30]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
# import cv2
# https://www.geeksforgeeks.org/ml-voting-classifier-using-sklearn/
import xgboost as xgb
from sklearn.ensemble import VotingClassifier
combined_features2 = np.concatenate([hogFeatures_np, histoFeatures_np,lbpFeatures_np], axis=1)
X_train, X_test, y_train, y_test = train_test_split(combined_features2, image_labels, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
rf_model = RandomForestClassifier(random_state=42, max_depth=20, min_samples_leaf=2, min_samples_split=10, n_estimators=300)
xgb_model = xgb.XGBClassifier(random_state=42, n_estimators=300, learning_rate=0.1, max_depth=6)
ensemble_model = VotingClassifier(estimators=[('rf', rf_model), ('xgb', xgb_model)], voting='soft')
ensemble_model.fit(X_train, y_train)
y_pred_ensemble = ensemble_model.predict(X_test)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f"Ensemble Model Accuracy: {accuracy_ensemble}")
print("Ensemble Classification Report:")
print(classification_report(y_test, y_pred_ensemble))



Ensemble Model Accuracy: 0.4142857142857143
Ensemble Classification Report:
                    precision    recall  f1-score   support

           calling       0.22      0.18      0.20       173
          clapping       0.45      0.39      0.42       160
           cycling       0.60      0.68      0.64       186
           dancing       0.54      0.54      0.54       180
          drinking       0.25      0.26      0.25       152
            eating       0.48      0.74      0.58       151
          fighting       0.52      0.53      0.52       186
           hugging       0.34      0.29      0.32       151
          laughing       0.43      0.42      0.43       179
listening_to_music       0.30      0.25      0.27       176
           running       0.46      0.45      0.46       155
           sitting       0.37      0.29      0.33       163
          sleeping       0.44      0.48      0.46       162
           texting       0.30      0.25      0.27       183
      using_laptop     

In [31]:
import pickle
import xgboost as xgb

with open('42_rf_xbBoost__hog_histo_lbp.pkl', 'wb') as file:
    pickle.dump(rf_model, file)